In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from CodeForDB import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.read_csv('aac_shelter_outcomes.csv')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Anthony Minunni\'s Project Two'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Hr(),
    html.Div(className='row',
             style={'display' : 'flex'},
                 children = [
                     html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                     html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                     html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                     html.Button(id='submit-button-three', n_clicks=0, children='Reset')
                  ],

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback(Output('datatable-interactivity',"data"),
              [Input('submit-button-one', 'n-clicks'), Input('submit-button-two', 'n-clicks'),
               Input('submit-button-three', 'n-clicks'), Input('submit-button-four', 'n-clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
    if(int(b1) == 0 and int(b2) == 0 and int(b3) == 0 and int(b4) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    elif(int(b1) >= int(b2) and int(b1) >= int(b3) and int(b1) >= int(b4)):
        df = pd.DaraFrame.from_records(list(shelter.read({"breed": {"$in":["Labrador Retriever Mix", 
                                                                         "Chesapeake Bay Retriever",
                                                                         "Newfoundland"]}},
                                                       {"sex_upon_outcome" : "Intact Female"},
                                                       {"age_upon_outcome_in_weeks": {"$gt": 26, "$lt": 156}})))
    elif(int(b2) >= int(b1) and int(b2) >= int(b3) and int(b2) >= int(b4)):
        df = pd.DaraFrame.from_records(list(shelter.read({"breed": {"$in":["German Shepherd", 
                                                                         "Alaskan Malamute", 
                                                                         "Old English Sheepdog",
                                                                         "Siberian Husky",
                                                                         "Rottweiler"]}}, 
                                                           {"sex_upon_outcome" : "Intact Male"},
                                                           {"age_upon_outcome_in_weeks": {"$gt": 26, "$lt": 156}})))               
    elif(int(b3) >= int(b1) and int(b3) >= int(b2) and int(b3) >= int(b4)):
         df = pd.DaraFrame.from_records(list(shelter.read({"breed": {"$in":["Doberman Pinsche", 
                                                                          "German Shepherd", 
                                                                          "Golden Retriever",
                                                                          "Bloodhound",
                                                                          "Rottweiler"]}}, 
                                                         {"sex_upon_outcome" : "Intact Male"},
                                                         {"age_upon_outcome_in_weeks": {"$gt": 20, "$lt": 300}})))
    elif(int(b4) >= int(b1) and int(b4) >= int(b2) and int(b4) >= int(b3)):
        df = pd.DataFrame.from_records(list(shelter.readAll({})))
                               
    return df.to_dict('records')


@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

'''@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    return [
        dcc.Graph( 
            df = px.DataFrame.from_dict(viewData)
            fig = px.pie(df, values='breed', names='day')
            fig.show()
        )    
    ] '''

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app